# 2a. DATA TRANSFORMATION & STANDARDIZATION

This notebook focuses on data cleaning operations:
1. **Text Normalization** - Convert to lowercase, fix encoding issues
2. **Typo Detection & Correction** - Fix special characters (caffÿ → caffè)
3. **Column Renaming** - Clean up problematic column names
4. **Address Consistency** - Verify and repair the Ubicazione field
5. **Data Wrangling** - Create macro-categories for exercise types

## 2a.1 Imports and Load Data

In [ ]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 140)

In [ ]:
# Load the original dataset
MILANO = pd.read_csv("Comune-di-Milano-Pubblici-esercizi(in)-2.csv", sep=";", encoding="utf-8")
print(f"Original shape: {MILANO.shape}")
MILANO.head()

---
# 1. TEXT NORMALIZATION

## 1.1 Convert Text Columns to Lowercase

In [ ]:
# Select text columns
text_cols = MILANO.select_dtypes(include="object").columns
print(f"Text columns: {list(text_cols)}")

In [ ]:
# Convert to lowercase
MILANO[text_cols] = MILANO[text_cols].apply(lambda col: col.str.lower())
MILANO.head()

## 1.2 Fix Column Names

In [ ]:
# The column 'þÿTipo esercizio storico pe' has encoding issues
print("Original columns:")
print(MILANO.columns.tolist())

In [ ]:
# Rename problematic columns
MILANO = MILANO.rename(columns={
    "þÿTipo esercizio storico pe": "Tipo esercizio storico pe",
    "Ubicazione": "Indirizzo",
    "Descrizione via": "Nome via"
})

print("Renamed columns:")
print(MILANO.columns.tolist())

---
# 2. TYPO DETECTION & CORRECTION

## 2.1 Detect Non-ASCII Characters

In [ ]:
# Find rows with problematic characters (ÿ, ý)
chars = ["ÿ", "ý"]
pattern = "|".join(map(lambda x: "\\" + x, chars))

mask = MILANO.select_dtypes(include="object").apply(
    lambda col: col.astype(str).str.contains(pattern, na=False)
).any(axis=1)

df_bad = MILANO[mask]
print(f"Rows with problematic characters: {len(df_bad)}")
df_bad.head()

## 2.2 Fix caffÿ → caffè

In [ ]:
# Count occurrences of 'caffÿ' pattern
count = 0
for c in MILANO.select_dtypes(include="object").columns:
    count += MILANO[c].astype(str).str.count(r"\bcaff[ÿý]").sum()

print(f"Occurrences of 'caffÿ/caffý' pattern: {count}")

In [ ]:
# Replace caffÿ/caffý with caffè
text_cols = MILANO.select_dtypes(include="object").columns
MILANO[text_cols] = MILANO[text_cols].apply(
    lambda col: col.str.replace(r"\bcaff[ÿý]", "caffè", regex=True)
)

# Also fix caffèý pattern
MILANO[text_cols] = MILANO[text_cols].apply(
    lambda col: col.str.replace(r"\bcaffè[ý]", "caffè", regex=True)
)

print("Fixed caffè pattern")

In [ ]:
# Remove remaining stray ÿ/ý characters from Indirizzo and Insegna
for col in ["Indirizzo", "Insegna"]:
    if col in MILANO.columns:
        MILANO[col] = MILANO[col].astype(str).str.replace("[ÿý]", "", regex=True)

print("Removed stray special characters")

In [ ]:
# Verify no more problematic characters remain
mask = MILANO.select_dtypes(include="object").apply(
    lambda col: col.astype(str).str.contains(pattern, na=False)
).any(axis=1)

print(f"Remaining rows with problematic characters: {mask.sum()}")

## 2.3 Check for Other Non-ASCII Characters

In [ ]:
# Pattern for non-ASCII excluding common Italian accented letters
pattern_non_ascii = r"[^\x00-\x7FàèéìòùÀÈÉÌÒÙ]"

mask = MILANO.select_dtypes(include="object").apply(
    lambda col: col.astype(str).str.contains(pattern_non_ascii, na=False)
).any(axis=1)

print(f"Rows with other non-ASCII characters: {mask.sum()}")
if mask.sum() > 0:
    display(MILANO[mask].head())

---
# 3. ADDRESS CONSISTENCY CHECK & REPAIR

Verify that the free-text `Indirizzo` field is consistent with structured address columns.

In [ ]:
# Work with address-related columns
addr_cols = ['Indirizzo', 'Tipo via', 'Nome via', 'Civico', 'Codice via', 'ZD']
ADDR = MILANO[addr_cols].copy()
ADDR.head()

## 3.1 Extract Components from Indirizzo

In [ ]:
# Regex patterns to extract components
REGEX_TIPO = r"^(.{3})"  # First 3 characters
REGEX_DESC = r"^.{3}\s+(.*?)\s+\b(?:N\.?|num\.)\b"  # Street name until N./num.
REGEX_CIV = r"\b(?:N\.?|num\.)\s*0*([0-9]+)"  # Civic number
REGEX_ZD = r"z\.d\.\s*(\d+)"  # Zone code

ADDR['ubi_tipo'] = ADDR['Indirizzo'].str.extract(REGEX_TIPO, expand=False)
ADDR['ubi_desc'] = ADDR['Indirizzo'].str.extract(REGEX_DESC, expand=False, flags=re.IGNORECASE)
ADDR['ubi_civico'] = ADDR['Indirizzo'].str.extract(REGEX_CIV, expand=False, flags=re.IGNORECASE)
ADDR['ubi_zd'] = ADDR['Indirizzo'].str.extract(REGEX_ZD, expand=False, flags=re.IGNORECASE)

ADDR[['Indirizzo', 'ubi_tipo', 'ubi_desc', 'ubi_civico', 'ubi_zd']].head(10)

## 3.2 Compare Extracted vs Structured Values

In [ ]:
# Build match flags
ADDR['Tipo_match'] = ADDR['ubi_tipo'] == ADDR['Tipo via']
ADDR['Descrizione_match'] = ADDR['ubi_desc'] == ADDR['Nome via']
ADDR['Civico_match'] = ADDR['ubi_civico'].astype(str) == ADDR['Civico'].astype(str)
ADDR['ZD_match'] = ADDR['ubi_zd'].astype(str) == ADDR['ZD'].astype(str)

# Summary
summary = pd.Series({
    'Total rows': len(ADDR),
    'Tipo mismatches': (~ADDR['Tipo_match']).sum(),
    'Descrizione mismatches': (~ADDR['Descrizione_match']).sum(),
    'Civico mismatches': (~ADDR['Civico_match']).sum(),
    'ZD mismatches': (~ADDR['ZD_match']).sum(),
})
summary

## 3.3 Selective Repair of Indirizzo

In [ ]:
# Rebuild Indirizzo from structured fields for rows with street name mismatch
ubi_rebuilt = (
    ADDR['Tipo via'].astype(str).str.strip() + ' ' +
    ADDR['Nome via'].astype(str).str.strip() + ' n. ' +
    ADDR['Civico'].astype(str).str.strip() + ' ' +
    '(z.d. ' + ADDR['ZD'].astype(str) + ')'
).str.replace(r"\s+", " ", regex=True).str.strip()

# Create clean Indirizzo column
ADDR['Indirizzo_clean'] = ADDR['Indirizzo']

# Only repair rows where street name doesn't match
mask_rebuild = ~ADDR['Descrizione_match']
ADDR.loc[mask_rebuild, 'Indirizzo_clean'] = ubi_rebuilt.loc[mask_rebuild]

print(f"Repaired {mask_rebuild.sum()} rows")

In [ ]:
# Validate repairs
ADDR['ubi_desc2'] = ADDR['Indirizzo_clean'].str.extract(REGEX_DESC, expand=False, flags=re.IGNORECASE)
ADDR['Descrizione_match_after'] = ADDR['ubi_desc2'] == ADDR['Nome via']

before = (~ADDR['Descrizione_match']).sum()
after = (~ADDR['Descrizione_match_after']).sum()

print(f"Descrizione mismatches BEFORE: {before}")
print(f"Descrizione mismatches AFTER: {after}")

In [ ]:
# Apply clean Indirizzo back to main dataframe
MILANO['Indirizzo'] = ADDR['Indirizzo_clean']
print("Applied cleaned Indirizzo to main dataframe")

## 3.4 Fill Missing Civico from Indirizzo

In [ ]:
# Check for null Civico
print(f"Null Civico before: {MILANO['Civico'].isna().sum()}")

# Fill from extracted ubi_civico where Civico is null
mask_fill_civico = MILANO['Civico'].isna() & ADDR['ubi_civico'].notna()
MILANO.loc[mask_fill_civico, 'Civico'] = ADDR.loc[mask_fill_civico, 'ubi_civico']

print(f"Null Civico after: {MILANO['Civico'].isna().sum()}")

---
# 4. DATA WRANGLING: CREATE MACRO-CATEGORIES

Create simplified macro-categories for exercise types based on the messy `Settore storico pe` column.

In [ ]:
# Define column names
settore_col = "Settore storico pe"
tipo_col = "Tipo esercizio storico pe"

# Normalize sector text
MILANO["settore_norm"] = MILANO[settore_col].astype(str).str.upper().fillna("")

In [ ]:
# Define patterns for each macro-category

# BAR
patterns_bar = [
    "BAR CAFFE", "CAFF", "BAR GASTRONOM", "BIRRERIA", "SALE DA BALLO",
    "BAR", "DISCO", "LOCALI NOTTURNI", "SPACCIO BEVANDE ANALCOLICHE",
    "GIOC", "SOMMINISTRAZIONE", "WINE", "PUB"
]

# PIZZERIA
patterns_piz = ["PIZZERIA", "PIZZERIE E SIMILI"]

# RISTORANTE
patterns_rist = ["RISTORA", "OSTERIA", "CUCINA", "TRATTORIA"]

# GASTRONOMIA
patterns_gast = [
    "GENERE MERCEOL", "PRODOTTI DI GASTRONOMIA", "PROD DI GASTRO",
    "TAVOLA FREDDA", "CIBI COTTI", "CIBI COTTI PRECONFEZIONATI",
    "MENSA", "TAVOLA CALDA", "TAV.CALDE,SELF SERVICE,FAST F",
    "SELF SERVICE", "FAST F"
]

# GELATERIA
patterns_gel = ["BAR PASTIC", "GELATERIA"]

In [ ]:
def build_mask(patterns):
    """Return boolean mask for rows containing any of the patterns."""
    regex = "|".join(patterns)
    return MILANO["settore_norm"].str.contains(regex, na=False)

mask_bar = build_mask(patterns_bar)
mask_piz = build_mask(patterns_piz)
mask_rist = build_mask(patterns_rist)
mask_gast = build_mask(patterns_gast)
mask_gel = build_mask(patterns_gel)

In [ ]:
# Assign macro-categories with hierarchy (RISTORANTE > PIZZERIA > BAR > GASTRONOMIA > GELATERIA)
MILANO["Tipo_macro"] = "ALTRO"

# Priority from lowest to highest
MILANO.loc[mask_gel, "Tipo_macro"] = "GELATERIA"
MILANO.loc[mask_gast, "Tipo_macro"] = "GASTRONOMIA"
MILANO.loc[mask_bar, "Tipo_macro"] = "BAR"
MILANO.loc[mask_piz, "Tipo_macro"] = "PIZZERIA"
MILANO.loc[mask_rist, "Tipo_macro"] = "RISTORANTE"  # Highest priority

# Check distribution
MILANO["Tipo_macro"].value_counts()

In [ ]:
# Clean up temporary column
MILANO = MILANO.drop(columns=["settore_norm"])
print("Created 'Tipo_macro' column with simplified categories")

---
# 5. SAVE TRANSFORMED DATASET

In [ ]:
# Summary of transformations applied
print("=== TRANSFORMATIONS APPLIED ===")
print("1. Converted all text columns to lowercase")
print("2. Renamed problematic column names")
print("3. Fixed caffÿ/caffý → caffè typos")
print("4. Removed stray special characters")
print("5. Repaired inconsistent Indirizzo field")
print("6. Filled missing Civico from Indirizzo")
print("7. Created Tipo_macro simplified categories")
print(f"\nFinal shape: {MILANO.shape}")

In [ ]:
# Save the transformed dataset
MILANO.to_csv("MILANO_transformed.csv", index=False, sep=";")
print("Saved: MILANO_transformed.csv")

In [ ]:
# Preview final dataset
MILANO.head()